![hopsworks_logo](../../images/hopsworks_logo.png)

# Part 02: Training Data & Feature views

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/fraud_batch/2_feature_view_creation.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store

## 🗒️ In this notebook we will see how to create a training dataset from the feature groups:
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![02_training-dataset](../../images/02_training-dataset.png)

#### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/164


Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;">⬇️ Data retrieving from Feature Groups</span>

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
fares_fg = fs.get_or_create_feature_group("fares_fg",
                                          version=1)
rides_fg = fs.get_or_create_feature_group("rides_fg",
                                          version=1)

In [3]:
# Select features for training data.
fg_query = fares_fg.select(['total_fare'])\
                            .join(rides_fg.select_except(['taxi_id', "driver_id", "pickup_datetime",
                                                          "pickup_longitude", "pickup_latitude",
                                                          "dropoff_longitude", "dropoff_latitude"]),
                                  on=['ride_id'])
fg_query.show(2)

2022-08-17 00:22:36,054 INFO: USE `romankah_featurestore`
2022-08-17 00:22:37,056 INFO: SELECT `fg1`.`total_fare` `total_fare`, `fg0`.`month_of_the_ride` `month_of_the_ride`, `fg0`.`ride_id` `ride_id`, `fg0`.`passenger_count` `passenger_count`, `fg0`.`distance` `distance`, `fg0`.`pickup_distance_to_jfk` `pickup_distance_to_jfk`, `fg0`.`dropoff_distance_to_jfk` `dropoff_distance_to_jfk`, `fg0`.`pickup_distance_to_ewr` `pickup_distance_to_ewr`, `fg0`.`dropoff_distance_to_ewr` `dropoff_distance_to_ewr`, `fg0`.`pickup_distance_to_lgr` `pickup_distance_to_lgr`, `fg0`.`dropoff_distance_to_lgr` `dropoff_distance_to_lgr`, `fg0`.`year` `year`, `fg0`.`weekday` `weekday`, `fg0`.`hour` `hour`
FROM `romankah_featurestore`.`fares_fg_1` `fg1`
INNER JOIN `romankah_featurestore`.`rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id`


,total_fare,month_of_the_ride,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,141.0,202001,1168,3,11.545955,19.488680,10.238375,27.825895,17.922794,12.224066,1.111962,2020,2,18
1,28.0,202001,36150,2,2.665704,8.748815,8.214922,15.657193,13.956273,2.604474,5.187167,2020,3,20


#### <span style="color:#ff5f27;">〰️ Transformation Functions</span>


We can preprocess our data using several encoding methods like *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [4]:
[t_func.name for t_func in fs.get_transformation_functions()]

['label_encoder', 'robust_scaler', 'min_max_scaler', 'standard_scaler']

In [5]:
# # Load transformation functions.
# min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
# label_encoder = fs.get_transformation_function(name="label_encoder")

# # Map features to transformations.
# transformation_functions = {
#     "total_fare": min_max_scaler,
#     "distance": min_max_scaler
# }

## <span style="color:#ff5f27;">🔮 Feature View Creation</span>


We start by selecting all the features we want to include for model training/inference.

After we have made a query from desired features, we should make a corresponding `Feature View`.
In order to do it we may use `fs.create_feature_view()`

In [6]:
nyc_fares_fv = fs.create_feature_view(
    name='nyc_taxi_fares_fv',
    query=fg_query,
    labels=["total_fare"]
#     transformation_functions=transformation_functions
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/164/fs/106/fv/nyc_taxi_fares_fv/version/1


In [7]:
nyc_fares_fv.version

1

## <span style="color:#ff5f27;">🏋️ Training Dataset Creation</span>
    
In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

Training Dataset may contain splits such as:

    Training set - the subset of training data used to train a model.
    Validation set - the subset of training data used to evaluate hparams when training a model
    Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using fs.create_train_validation_test_split() method.

In [8]:
nyc_fares_fv.create_training_data(
    description='training_dataset',
    data_format='csv'
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/nyc_taxi_fares_fv_1_1_create_fv_td_16082022222340/executions


(1, <hsfs.core.job.Job at 0x24ad5dc2d60>)

In [9]:
nyc_fares_fv.create_train_test_split(
    test_size=0.2 # here we can define the test dataset size
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/nyc_taxi_fares_fv_1_2_create_fv_td_16082022222437/executions


(2, <hsfs.core.job.Job at 0x24ad5d97070>)

In [10]:
X_train, y_train = nyc_fares_fv.get_training_data(
    training_dataset_version=1
)

In [11]:
X_train.head(5)

,month_of_the_ride,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,197001,41172,4,37.799183,53.517080,23.520740,72.729666,44.407565,56.187501,32.117385,1970,0,14
1,197001,41082,2,48.831464,31.249725,31.191905,46.971877,14.674872,30.137979,22.458636,1970,0,13
2,197001,41146,1,53.347155,45.430518,63.451505,33.338823,69.867283,34.988519,56.629800,1970,0,14
3,197001,41116,1,48.478259,32.318731,57.715072,53.281089,69.707493,38.885463,53.899009,1970,0,13
4,197001,41147,2,2.710774,7.468345,10.161852,13.581373,10.934313,8.804826,9.918372,1970,0,13


In [12]:
y_train.head(5)

,total_fare
0,111.0
1,148.0
2,230.0
3,27.0
4,30.0


In [13]:
X_train, y_train, X_test, y_test = nyc_fares_fv.get_train_test_split(
    training_dataset_version=2
)

In [14]:
X_test.head(5)

,month_of_the_ride,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,202001,7938,2,3.382435,8.231768,9.140160,13.398873,11.948914,6.174618,9.533101,2020,4,8
1,202001,13879,2,2.138916,9.228437,10.383209,11.888775,11.433648,9.728798,11.859925,2020,5,17
2,202001,7243,2,4.727096,8.210070,9.587711,13.622313,11.690100,5.767397,10.461807,2020,4,4
3,202001,8383,2,8.496722,8.213907,11.578282,13.561513,19.546400,5.876488,2.837954,2020,4,10
4,202001,10348,2,1.809504,8.226930,8.299477,13.433025,14.481473,6.111485,4.330426,2020,4,21


In [15]:
y_test.head(5)

,total_fare
0,3.0
1,3.0
2,4.0
3,4.0
4,4.0


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the next notebook, we will train a model on the dataset we created in this notebook.